In [1]:
number_of_shots = 3

In [2]:
experiment_name = "2024.05.07-01_Mixtral_7B_instruct_API_3-few-shots_RANDOM"

In [3]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [4]:
!pip install -q openai #--progress-bar off

In [5]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [6]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [7]:
# output_datapath = "data/output/2023.11.16-1_Mistral_7B_instruct_API/"
# final_report_name = 'report_sdg_classification_Mistral_7B.xlsx'
output_datapath = f"data/output/{experiment_name}/"
log_filename = f"logs/{experiment_name}.log"

final_report_name = 'report_sdg_classification_Mixtral_7B.xlsx'

In [8]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [9]:
data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx").drop(columns=['Unnamed: 0'])
data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
    (data_sel_4_few_shot_examples.text_id==5614) |
    (data_sel_4_few_shot_examples.text_id==2089) |
    (data_sel_4_few_shot_examples.text_id==5665) #|
#     (data_sel_4_few_shot_examples.text_id==370)  |
#     (data_sel_4_few_shot_examples.text_id==1422) |
#     (data_sel_4_few_shot_examples.text_id==4329) |
#     (data_sel_4_few_shot_examples.text_id==4929) 

]

data_sel_4_few_shot_examples

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
2319,2319,5665,10.18356/3fe10a08-en,"As a result, both the trade and current accoun...",8,0.0,5.0,1.0000,53
3569,3569,2089,10.18356/ee52a573-en,"This refers to the manufacture, use and dispos...",12,3.0,29.0,0.8125,47
4608,4608,5614,10.2139/SSRN.1406042,The modern definitions of the inchoate offence...,16,0.0,6.0,1.0000,136


In [10]:
few_shots = ""

for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
    input_text = "Input text: \""+row['text']+"\"\n"
    output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'
    
    few_shots += input_text
    few_shots += output_text
    
print(few_shots)

Input text: "As a result, both the trade and current account deficits increased in 2012. Large current account deficits, despite the slowdown in economic growth, are symptomatic of demand-supply imbalances and a pointer to the urgent need to resolve supply bottlenecks. However, capital flows have been adequate to cover the current account deficit thus far."
Output: "---SDG 8---"

Input text: "This refers to the manufacture, use and disposal of lifestyle goods such as electronics and clothing. Key sectors of high impact include food, housing and transport. In these sectors, wafer, energy and waste can be considered as crosscutting sectors affecting and affected by almost every lifestyle domain."
Output: "---SDG 12---"

Input text: "The modern definitions of the inchoate offences are well known in the criminal law since the Middle Ages both in European-Continent legal systems and in the English Common Law. They were mostly developed by the case-laws of the Star Chamber Court, which was a

In [11]:
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx")
data = data.drop(columns=['Unnamed: 0'])
data.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125


In [12]:
# template_part1 = """[INST]
# <<SYS>>
# In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The text to be processed is:
# <</SYS>>

# '''
# """

# template_part2 = """
# '''

# <<SYS>>
# Answer with the label only using the format:---LABEL---.
# <</SYS>>
# [/INST]
# """

system_prompt = f"""In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "---SDG 0---".

The following are {number_of_shots} examples of expected classification given an input text:

"""+few_shots+"""

Answer with the label only using the format:---LABEL---
"""


In [13]:
data_sel = data #.iloc[:5]

In [15]:
import time

# cnt = 0
results = []
for index, row in data_sel.iterrows():
    
    cnt = str(index)
    
    file = open(log_filename, 'a+', encoding='utf-8')
    file.write("\n\n##################################################\n")
    
    text_id = row['text_id']
    text = row["text"]
    sdg_true = row["sdg"]
    
    #query = template_part1 + text + template_part2
    query = text
    
    try:
        body = {
          "model": model_name,
          "messages": [
              {"role": "system", "content": system_prompt}, 
              {"role": "user", "content": query}],
          "temperature": 0.0
        }

        result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
        print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
        file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+str(result)+"\n")
    except Exception as e:
        result = str(e)
        print(cnt, text_id, sdg_true, "ERROR: "+result)
        file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+" "+"ERROR: "+result+"\n")        
    
#     print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
    
    result_extended = (cnt, text_id, sdg_true, result)
    #print(result_extended)
    
    with open(output_datapath+f'sdg_classification_{model_name.replace("/","_")}_{cnt}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)
    
    results.append(result_extended)
    #cnt+=1
    
    file.close()
    
    time.sleep(0.5)
    
    

0 6668 0  ---SDG 11---
1 6099 0  ---SDG 11--- Sustainable Cities and Communities; The text highlights a model shoot in the Maldives, which can be categorized under Sustainable Cities and Communities, as it showcases the natural beauty of the location and supports sustainable tourism.
2 6471 0  ---SDG 16---

The text discusses the legal proceedings related to the Aurangabad arms haul case in India, specifically the suspension of a life sentence and granting of conditional bail to a convict named Afroz Khan Pathan. It also mentions the lack of prima facie evidence to support the claim that Pathan visited Bangladesh for terror activities. This is related to SDG 16, which aims to promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels.
3 1786 0  ---SDG 4--- (Goal 4: Quality Education) as the text refers to the naming of minor planets after Indian scientists, including a

56 2463 0  ---SDG 8---

The text discusses employment and career advancement in the field of sports management, which can be linked to Decent Work and Economic Growth (SDG 8).
57 193 0  ---SDG 9--- (Industry, Innovation and Infrastructure) as the text discusses the progress and development of the Ariane 6 space launch vehicle, which falls under the category of infrastructure development and technological innovation.
58 3497 0  ---SDG 3---
59 3751 0  ---SDG 3--- (Goal 3 is "Good Health and Well-being" in the SDGs, and the text seems to be about managing Type 2 diabetes through exercise, which falls under this goal.)
60 6573 1  ---SDG 1---

Explanation:

The text is about poverty and deprivation, which are directly related to SDG 1: No Poverty. This goal aims to end poverty in all its forms everywhere. The text mentions different studies and concepts related to poverty and deprivation, which are crucial in understanding and addressing poverty. Therefore, the text can be classified as SDG

103 4534 1  ---SDG 1---	
Explanation:
The text provided discusses the concept of poverty and the poverty line, which is directly related to SDG 1: No Poverty. The text mentions Rowntree's 'poverty line' and the division of those in poverty into two groups: primary poverty and secondary poverty. This classification highlights the issue of income inequality and the struggle to meet basic needs, which are key targets of SDG 1.
104 1733 1  ---SDG 8---
105 2694 1  ---SDG 1---

Explanation:

The text provided discusses the decomposition of changes in child poverty, which is directly related to SDG 1: No Poverty. The decomposition helps to measure the portion of changes in child poverty due to changes in the characteristics of the child population and those due to changes in the association between these characteristics and poverty rates. The text also mentions the role of social transfers in responding to changes in labor income and how it varies across the income distribution. This focus on

158 654 2  ---SDG 2---
159 4178 2  ---SDG 2---
160 5812 2  ---SDG 2---
161 5130 2  ---SDG 2---
162 4078 2  ---SDG 8---
163 646 2  ---SDG 2--- (Sustainable Development Goal 2 is Zero Hunger, which includes targets to end all forms of hunger and malnutrition by 2030)
164 4120 2  ---SDG 2---
165 894 2  ---SDG 2---
(This classification is given because the text emphasizes the importance of improving local food production, addressing food security issues, and enhancing rural livelihoods, all of which are aligned with the objectives of Sustainable Development Goal 2 - Zero Hunger.)
166 1114 2  ---SDG 2---
167 211 2  ---SDG 2---
168 4039 2  ---SDG 2---
(SDG 2 aims to "End hunger, achieve food security and improved nutrition and promote sustainable agriculture.")
169 4054 2  ---SDG 9---
170 1005 2  ---SDG 2---
171 471 2  ---SDG 2--- (Sustainable Development Goal 2: Zero Hunger)
172 362 2  ---SDG 5---
173 3582 2  ---SDG 2---
174 4485 2  ---SDG 2---
175 6144 2  ---SDG 12---
176 4757 2  ---SDG 13

259 3447 4  ---SDG 4---
260 3865 4  ---SDG 4---
261 932 4  ---SDG 4---
262 6608 4  ---SDG 8---
263 1276 4  ---SDG 4--- (Goal 4: Quality Education)
264 2037 4  ---SDG 4---
265 5181 4  ---SDG 4--- (Quality Education)
266 2261 4  ---SDG 4---
267 6446 4  ---SDG 9---
268 2771 4  ---SDG 4---
269 991 4  ---SDG 4---
270 1449 4  ---SDG 4--- (Quality Education)
271 3632 4  ---SDG 4---
272 3908 4  ---SDG 4---
273 6580 4  ---SDG 4---
274 554 4  ---SDG 8---
275 5343 4  ---SDG 4---
276 1547 4  ---SDG 4---
277 202 4  ---SDG 4---
278 863 4  ---SDG 4--- (Quality Education)
279 3392 4  ---SDG 4---
280 2726 4  ---SDG 4--- (Quality Education)
281 3429 4  ---SDG 8--- (Decent Work and Economic Growth) as the text discusses the importance of utilizing the skills of international students for economic growth and development.
282 4225 4  ---SDG 4--- (Goal 4: Quality Education)
283 4789 4  ---SDG 4---
284 2661 4  ---SDG 4--- (Quality Education)
285 736 4  ---SDG 4--- (Goal 4: Quality Education)
286 4116 4  ---S

...

845 4721 14  ---SDG 14--- (Sustainable Goal 14 is "Life Below Water" and the text discusses the reduction of pressures on global fisheries and the expansion of seafood exports, which aligns with the goal of sustainable use of marine resources.)
846 6196 14  ---SDG 14---
847 432 14  ---SDG 14--- (Sustainable use of marine resources)
848 4876 14  ---SDG 14---
849 785 14  ---SDG 14--- (Life Below Water)
850 3619 14  ---SDG 14--- (Life Below Water) as the text discusses the importance of rebuilding and effectively managing fisheries to ensure sustainable fishing and prevent overfishing, which aligns with SDG 14 target 14.4.
851 4158 14  ---SDG 14--- (Life Below Water)
852 5523 14  ---SDG 14---
853 1012 14  ---SDG 14--- (Sustainable Development Goal 14: Life Below Water)
854 1453 14  ---SDG 14---
855 6496 14  ---SDG 14---
856 843 14  ---SDG 14--- (This example focuses on fishing and fishery, which aligns with SDG 14's objectives of sustainable use of marine resources.)
857 4311 14  ---SDG 1

924 5980 15  ---SDG 15---
925 2985 15  ---SDG 15--- (Life on Land) is the most appropriate goal for this text, as it focuses on the study and preservation of various plant and animal species, including endemic and subendemic ones, in Kyrgyzstan.
926 1515 15  ---SDG 13--- (Climate Action) as the text discusses the impact of roundwood harvesting in Finland on CO2 emissions and carbon sinks, which relates to climate change mitigation efforts.
927 6479 15  ---SDG 15---
928 2296 15  ---SDG 15--- (Goal 15 is about Life on Land, which includes targets for protecting, restoring and promoting the sustainable use of terrestrial ecosystems, sustainable forest management, combating desertification, and halting and reversing land degradation and halting biodiversity loss.)
929 3610 15  ---SDG 15---
930 152 15  ---SDG 15---

Explanation:
The text discusses issues related to the management and protection of biodiversity, specifically in the context of poor coordination among sectoral strategies, over

966 930 16  ---SDG 16---
967 4430 16  ---SDG 16---
(Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels) is the most fitting goal for the given text. The text discusses the legitimacy of judicial review of a constitution's federal and structural provisions, which is directly related to building effective and inclusive institutions.
968 2761 16  ---SDG 16---
(This answer is based on the assumption that the text is discussing the use of electronic government as a tool for improving governance, transparency, and the rule of law, which aligns with SDG 16.)
969 3839 16  ---SDG 16---
The text discusses the role of sensational jurisprudence, which is a new branch of sociolegal studies dealing with the five senses and public policy. The focus of the essay is on social science research about images of suffering and the implications of these findings, particularly in the 

992 2469 16  ---SDG 16---
(Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels) is the most appropriate goal for this text, as it discusses legislative changes in Australia related to the use of lethal force to prevent terrorist attacks, which falls under the category of building effective and accountable institutions for promoting justice and peace.
993 1609 16  ---SDG 17---
(This goal was chosen because the text discusses public policy and administration, which can be linked to SDG 17's focus on strengthening global partnerships and cooperation for sustainable development.)
994 5916 16  ---SDG 16---
995 567 16  ---SDG 16---

The text discusses issues related to State Surveillance, Censorship, and Internet Intermediaries practices, which can be classified under SDG 16: Peace, Justice and Strong Institutions. Specifically, it highlights the importance of these i

1019 1947 16  ---SDG 16---

The text discusses the emergence of new social actors, particularly nongovernmental organizations (NGOs), in Latin American civil society and the debates around their role in social and political democratization. This topic relates to SDG 16, which promotes peaceful and inclusive societies for sustainable development, provides access to justice for all and builds effective, accountable and inclusive institutions at all levels. The discussion of NGOs and their impact on civil society is relevant to the promotion of accountable and inclusive institutions, which is a key aspect of SDG 16.


In [16]:
files = glob.glob(output_datapath + "*.pkl")
# files

In [36]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    if len(res[3]['choices'])>1:
        raise Exception('ERROR: Multilabel detected!')
    sdg_pred = res[3]['choices'][0]['message']['content']
    res4df = (res[0], res[1], res[2], sdg_pred)
    print(res4df)
    tuple_list.append(res4df)

('1015', 4695, 16, ' ---SDG 16---')
('582', 714, 9, ' ---SDG 9--- (Industry, innovation and infrastructure)')
('2', 6471, 0, ' ---SDG 16---\n\nThe text discusses the legal proceedings related to the Aurangabad arms haul case in India, specifically the suspension of a life sentence and granting of conditional bail to a convict named Afroz Khan Pathan. It also mentions the lack of prima facie evidence to support the claim that Pathan visited Bangladesh for terror activities. This is related to SDG 16, which aims to promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels.')
('782', 2180, 13, ' ---SDG 11--- (Sustainable Cities and Communities)')
('601', 4947, 10, ' ---SDG 10---')
('144', 2727, 2, ' ---SDG 2---\n\nThe text discusses the need for a more radical and integrated policy approach to promote sustainable food production and reduce poverty, hunger, and environmen

In [37]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [38]:
results_processed

,cnt,id,true_label,predicted_label
0,1015,4695,16,---SDG 16---
1,582,714,9,"---SDG 9--- (Industry, innovation and infrast..."
2,2,6471,0,---SDG 16---\n\nThe text discusses the legal ...
3,782,2180,13,---SDG 11--- (Sustainable Cities and Communit...
4,601,4947,10,---SDG 10---
...,...,...,...,...
1015,152,5700,2,---SDG 1---\n\nExplanation:\nThe text discuss...
1016,276,1547,4,---SDG 4---
1017,554,5840,9,---SDG 8---
1018,927,6479,15,---SDG 15---


In [39]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 11---
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 11--- Sustainable Cities and Communiti...
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---\n\nThe text discusses the legal ...
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146,3,1786,0,---SDG 4--- (Goal 4: Quality Education) as th...
4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124,4,2143,0,---SDG 7---
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100,1015,4695,16,---SDG 16---
1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60,1016,6628,16,---SDG 17---\n\n(Note: I classified the text ...
1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124,1017,4563,16,---SDG 16---
1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109,1018,6710,16,---SDG 16---\n\nThis text discusses legal iss...


In [40]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,predicted_label_clean
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 11---,11
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 11--- Sustainable Cities and Communiti...,11
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---\n\nThe text discusses the legal ...,16
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146,3,1786,0,---SDG 4--- (Goal 4: Quality Education) as th...,4
4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124,4,2143,0,---SDG 7---,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100,1015,4695,16,---SDG 16---,16
1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60,1016,6628,16,---SDG 17---\n\n(Note: I classified the text ...,17
1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124,1017,4563,16,---SDG 16---,16
1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109,1018,6710,16,---SDG 16---\n\nThis text discusses legal iss...,16


In [41]:
results_processed_merged.groupby("predicted_label_clean").count()

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,,,
0,14,14,0,14,14,0,0,0,14,14,14,14,14
1,67,67,67,67,67,67,67,67,67,67,67,67,67
10,29,29,29,29,29,29,29,29,29,29,29,29,29
11,86,86,76,86,86,76,76,76,86,86,86,86,86
12,54,54,53,54,54,53,53,53,54,54,54,54,54
13,63,63,62,63,63,62,62,62,63,63,63,63,63
14,58,58,58,58,58,58,58,58,58,58,58,58,58
15,59,59,57,59,59,57,57,57,59,59,59,59,59
16,51,51,47,51,51,47,47,47,51,51,51,51,51


In [42]:
results_processed_merged.to_excel(output_datapath+final_report_name)

In [43]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       1.00      0.23      0.38        60
           1       0.73      0.82      0.77        60
           2       0.96      0.80      0.87        60
           3       0.79      0.93      0.85        60
           4       0.84      0.90      0.87        60
           5       0.85      0.97      0.91        60
           6       0.98      0.87      0.92        60
           7       0.90      0.92      0.91        60
           8       0.46      0.73      0.57        60
           9       0.73      0.75      0.74        60
          10       0.86      0.42      0.56        60
          11       0.69      0.98      0.81        60
          12       0.89      0.80      0.84        60
          13       0.81      0.85      0.83        60
          14       0.97      0.93      0.95        60
          15       0.90      0.88      0.89        60
          16       0.90      0.77      0.83        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
# AGGIUNTO IL 2024.05.07:

In [50]:
# CORREZIONI NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI O OUTPUT NON UTILI PER DEFINIRE LA CLASSE O CLASSI NON PREVISTE 

import random

# NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI SELEZIONO SEMPRE IL PRIMO
results_processed_merged = results_processed_merged.drop_duplicates(subset='text_id', keep='first')

# # NEL CASO IN CUI IL MODELLO RESTITUISCA OUTPUT NON UTILI PER DEFINIRE LA CLASSE SI APPLICA LA CLASSE 0
# # results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = 0
# results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'sdg'] - 1

# # NEL CASO IN CUI IL MODELLO RESTITUISCA CLASSI NON PREVISTE SI APPLICA LA CLASSE 0
# results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if int(x)>16 else x)

# Lista di valori sdg_list
sdg_list = [str(x) for x in range(17)]
# sdg_list = [x for x in range(17)]
# print(sdg_list)

# Se il valore di 'predicted_label_clean' e' NaN o un valore numerico non presente nella lista sdg_list 
# allora assegna a  'predicted_label_clean' un valore qualunque estratto a caso da sdg_list ma diverso dal valore di 'sdg' per la riga in esame
# Modifica della colonna 'predicted_label_clean' secondo le regole specificate
for index, row in results_processed_merged.iterrows():
    if pd.isna(row['predicted_label_clean']) or row['predicted_label_clean'] not in sdg_list:
#         new_value = random.choice([x for x in sdg_list if x != row['sdg']])
        results_processed_merged.at[index, 'predicted_label_clean'] = 0  # new_value

In [51]:
results_processed_merged.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,predicted_label_clean
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 11---,11
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 11--- Sustainable Cities and Communiti...,11
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---\n\nThe text discusses the legal ...,16


In [52]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.52      0.25      0.34        60
           1       0.73      0.82      0.77        60
           2       0.96      0.80      0.87        60
           3       0.79      0.93      0.85        60
           4       0.84      0.90      0.87        60
           5       0.85      0.97      0.91        60
           6       0.98      0.87      0.92        60
           7       0.90      0.92      0.91        60
           8       0.46      0.73      0.57        60
           9       0.73      0.75      0.74        60
          10       0.86      0.42      0.56        60
          11       0.69      0.98      0.81        60
          12       0.89      0.80      0.84        60
          13       0.81      0.85      0.83        60
          14       0.97      0.93      0.95        60
          15       0.90      0.88      0.89        60
          16       0.90      0.77      0.83        60

    accuracy    

In [53]:
classification_report_filename = output_datapath+f'/{experiment_name}_classification_report.xlsx'
classification_report_filename

report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100
report_df.to_excel(classification_report_filename)

In [54]:
report_df

,precision,recall,f1-score,support
0,51.724138,25.000000,33.707865,60.000000
1,73.134328,81.666667,77.165354,60.000000
2,96.000000,80.000000,87.272727,60.000000
3,78.873239,93.333333,85.496183,60.000000
4,84.375000,90.000000,87.096774,60.000000
5,85.294118,96.666667,90.625000,60.000000
6,98.113208,86.666667,92.035398,60.000000
7,90.163934,91.666667,90.909091,60.000000
8,46.315789,73.333333,56.774194,60.000000
9,72.580645,75.000000,73.770492,60.000000


In [55]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [56]:
print(precision, recall, accuracy, f1_score)

81.04738403417414 79.80392156862746 79.80392156862746 79.16858204164399


In [57]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))

81.0 79.8 79.8 79.2
